In [1]:
import numpy as np

In [2]:
def padding_sequence(seq, max_len = 501, repkey = 'N'):
    seq_len = len(seq)
    if seq_len < max_len:
        gap_len = max_len -seq_len
        new_seq = seq + repkey * gap_len
    else:
        new_seq = seq[:max_len]
    return new_seq

In [3]:
def get_RNA_seq_concolutional_array(seq, motif_len = 4):
    seq = seq.replace('U', 'T')
    alpha = 'ACGT'
    #for seq in seqs:
    #for key, seq in seqs.iteritems():
    row = (len(seq) + 2*motif_len - 2)
    new_array = np.zeros((row, 4))
    for i in range(motif_len-1):
        new_array[i] = np.array([0.25]*4)
    
    for i in range(row-3, row):
        new_array[i] = np.array([0.25]*4)
        
    #pdb.set_trace()
    for i, val in enumerate(seq):
        i = i + motif_len-1
        if val not in 'ACGT':
            new_array[i] = np.array([0.25]*4)
            continue
        #if val == 'N' or i < motif_len or i > len(seq) - motif_len:
        #    new_array[i] = np.array([0.25]*4)
        #else:
        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except:
            pdb.set_trace()
        #data[key] = new_array
    return new_array


In [4]:
def get_bag_data_1_channel(data, max_len = 501):
    bags = []
    seqs = data["seq"]
    labels = data["Y"]
    for seq in seqs:
        #pdb.set_trace()
        #bag_seqs = split_overlap_seq(seq)
        bag_seq = padding_sequence(seq, max_len = max_len)
        #flat_array = []
        bag_subt = []
        #for bag_seq in bag_seqs:
        tri_fea = get_RNA_seq_concolutional_array(bag_seq)
        bag_subt.append(tri_fea.T)

        
        bags.append(np.array(bag_subt))
    
        
    return bags, labels

In [5]:
def read_seq_graphprot(seq_file, label = 1):
    seq_list = []
    labels = []
    seq = ''
    with open(seq_file, 'r') as fp:
        for line in fp:
            if line[0] == '>':
                name = line[1:-1]
            else:
                seq = line[:-1].upper()
                seq = seq.replace('T', 'U')
                seq_list.append(seq)
                labels.append(label)
    
    return seq_list, labels

In [6]:
def read_data_file(posifile, negafile = None, train = True):
    data = dict()
    seqs, labels = read_seq_graphprot(posifile, label = 1)
    if negafile:
        seqs2, labels2 = read_seq_graphprot(negafile, label = 0)
        seqs = seqs + seqs2
        labels = labels + labels2
        
    data["seq"] = seqs
    data["Y"] = np.array(labels)
    
    return data

In [7]:
def get_data(posi, nega = None, channel = 7,  window_size = 101, train = True):
    data = read_data_file(posi, nega, train = train)
    if channel == 1:
        train_bags, label = get_bag_data_1_channel(data, max_len = window_size)

    else:
        train_bags, label = get_bag_data(data, channel = channel, window_size = window_size)
    
    return train_bags, label

In [8]:
f, l = get_data('./CLIPSEQ_AGO2.train.positives.fa', './CLIPSEQ_AGO2.train.negatives.fa', channel=1)

In [9]:
f_test, l_test = get_data('./CLIPSEQ_AGO2.ls.positives.fa','CLIPSEQ_AGO2.ls.negatives.fa', channel=1, train=False)

In [10]:
f = np.array(f)
f = np.swapaxes(f, -1, 1)

In [11]:
f_test = np.array(f_test)
f_test = np.swapaxes(f_test, -1, 1)

In [12]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Using TensorFlow backend.


In [13]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(10, 3),
                 activation='relu',
                 input_shape=f.shape[1:], padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(128, (10, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(256, (5, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(Dropout(0.25))
# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 107, 4, 128)       3968      
_________________________________________________________________
dropout_1 (Dropout)          (None, 107, 4, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 4, 128)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 35, 4, 128)        163968    
_________________________________________________________________
dropout_2 (Dropout)          (None, 35, 4, 128)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 4, 128)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 4, 256)        164096    
__________

In [14]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.fit(f, l,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(f_test, l_test))

Train on 92346 samples, validate on 1000 samples
Epoch 1/50
92346/92346 [==============================] - 10s 112us/step - loss: 0.6875 - acc: 0.5432 - val_loss: 0.6794 - val_acc: 0.5720
Epoch 2/50
92346/92346 [==============================] - 8s 86us/step - loss: 0.6789 - acc: 0.5752 - val_loss: 0.6701 - val_acc: 0.6140
Epoch 3/50
92346/92346 [==============================] - 8s 86us/step - loss: 0.6727 - acc: 0.5904 - val_loss: 0.6615 - val_acc: 0.6340
Epoch 4/50
92346/92346 [==============================] - 8s 85us/step - loss: 0.6663 - acc: 0.5998 - val_loss: 0.6530 - val_acc: 0.6180
Epoch 5/50
92346/92346 [==============================] - 8s 86us/step - loss: 0.6591 - acc: 0.6053 - val_loss: 0.6554 - val_acc: 0.6380
Epoch 6/50
92346/92346 [==============================] - 8s 86us/step - loss: 0.6520 - acc: 0.6112 - val_loss: 0.6316 - val_acc: 0.6220
Epoch 7/50
92346/92346 [==============================] - 8s 85us/step - loss: 0.6472 - acc: 0.6131 - val_loss: 0.6295 - val_ac